In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np
from scipy.stats import pearsonr
#import image_analysis.plot_cells
#from image_analysis.postprocess_data_analysis import *
import seaborn as sns
import datetime

#!pip install nardini
from localcider.sequenceParameters import SequenceParameters
#from nardini.score_and_plot import calculate_zscore
from Bio import SeqIO
from io import StringIO
#from nardini.constants import NUM_SCRAMBLED_SEQUENCES, TYPEALL, TYPEALL_9x9
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import statsmodels.stats.multitest
import sys
#sys.path.append('/Users/kkappel/src/iupred2a')
#import iupred2a_lib
import matplotlib as mpl
from scipy.stats import linregress
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn import linear_model
import random
from matplotlib.colors import LinearSegmentedColormap
from Bio.Align import PairwiseAligner
from scipy.cluster.hierarchy import linkage, dendrogram
from Bio.Align import PairwiseAligner
from Bio import pairwise2
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\faris\anaconda3\Lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
# get mobidb sequences:
# these are all predicted disordered regions in the human proteome
# generate this mobid file by running 'bash get_mobid_data.sh'

mobidb_disordered_regions = []
for line in open('mobidb_disorder_preds_and_seqs.tsv'):
    if 'sequence' in line:
        continue
    accid = line.split('\t')[0]
    seq = line.split('\t')[1]
    regions = line.split('\t')[3]
    seq_len_listed = int(line.split('\t')[-1])
    if len( seq ) != seq_len_listed:
        print( "wrong length", accid )
    for subr in regions.split(','):
        subr_start = int(subr.split('..')[0])
        subr_end = int(subr.split('..')[1])
        if (subr_end - subr_start) < 30: continue
        subseq = seq[subr_start-1:subr_end]
        if len( subseq ) != (subr_end - subr_start + 1):
            print( accid )
            print( seq )
            print( seq_len_listed )
            print( subr )
            continue
        mobidb_disordered_regions.append( subseq )

C9JWL8
MTPALTALLCLGLSLGPRTRMQAGPFPKPTLWAEPGSVISWGSPVTIWCQGSLEAQEYQLDKEGSPEPWDRNNPLEPKNKARFSIPSMTQHHAGRYRCHYYSSAGWSEPSDPLELVMTGFYNKPTLSALPSPVVASGGNMTLRCGSQKGYHHFVLMKE
158
472..631
H0YJ05
XEPVMRVTLGTAKYEHAFNSIVWRINRLPDKNSASGHPHCFFCHLELGSDREVPSRFANHVNVEFSMPTTSASKASVRSISVEDKTDVRKWVNYSAHYSYQVEIEQKKSLKPDFEGDEMENPKECGVQ
128
144..282
A0A0A0MRP4
MHGRAYLLLHRDFCDLKENNYKGITAKPVSEDMMEWEVEIEGLQNSVWQGLVFQLTIHFTSEYNYAPPVVKFITIPFHPNVDPHTGQPCIDFLDNPEKWNTNYTLSSILLALQVMLSNPVLENPVNLEAARILVKDESLYRTILRLFNRPLQMKDDSQELPKDPRKCIRPIKTTSFSDYYQTWSRIATSKATEYYRTPLLKVPNFIGQYYKWKKMDLQHQKEWNLKYSVIKCWLARKRMPHEVTHSMEEIKLCPTL
256
281..317
G3V3A2
MWTGGRRPGRLRRAASAADMEKLSALQEQKGELRKRLSYTTHKLEKLETEFDSTRHYLEIELRRAQEELEKVTEKLRRIQSNYMALQRINQELEDKLYRMGQHYEEEKRALSHEIVALNSHLLEAKVTIDKLSEDNELYRKDCNLAAQLLQCSQTYGRVHKVSELPSDF
169
518..612
Q5JRP2
MAARPLPVSPARALLLALAGALLAPCEARGVSLWNQGRADEVVSASVGSGDLWIPVKSFDSKNHPEVLNIRLQRESKELIINLERNEGLIASSFTETHYLQDGTDVSLARNYTGHCYYHGHVRGYSDSAVSLSTCSGLRGLIVFENESYVLEPMKSATNRYKLFPAKKLKSVRGSCGSHHNTPNLAAKNVFPPP

In [4]:
mobid_df = pd.read_csv( 'mobidb_disorder_preds_and_seqs.tsv', delimiter='\t' )

In [12]:
# REPLACE WITH YOUR CSV
df = pd.read_csv("ZincFinger_Classical_RBD.csv")

C:\Users\faris\AppData\Local\Temp\ipykernel_2784\2181719650.py:1: DtypeWarning: Columns (11,12,13,1134,1138,1146,1154,1158,1162,1166,1170,1174,1178,1182,1186,1190,1194,1206,1214,1218,1222,1226,1230,1234,1238,1242,1250,1254,1258,1262,1266,1270,1274,1278,1282,1286,1290,1294,1298,1302,1306,1310,1314,1318,1332,1336,1340,1348,1352,1356,1360,1364,1376,1380,1384,1392,1408,1412,1428) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ZincFinger_Classical_RBD.csv")


In [13]:
cols_to_merge = ["acc", "start..end", "content_fraction", "content_count", "length"]

In [ ]:
df = pd.merge(df, mobid_df[cols_to_merge], left_on = 'uniprot_id', right_on = 'acc', how = 'left')

In [ ]:
df.drop("acc", axis=1, inplace=True)

In [ ]:
num_disordered_regions = df["start..end"].apply(lambda x: len(x.split("..")) - 1 if isinstance(x,str) else 0)

In [ ]:
df["number_disordered_regions"] = num_disordered_regions

In [ ]:
def find_average_length(all_start_ends):
    if not isinstance(all_start_ends, str):
        return 0
    seqs = all_start_ends.split(",")
    total = 0
    for seq in seqs:
        start_end = seq.split("..")
        start_end_num = list(map(int, start_end))
        diff = abs(start_end_num[1] - start_end_num[0])
        total += diff
    return total / len(seqs)

In [ ]:
df["average_disordered_region_size"] = df["start..end"].apply(find_average_length)